In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

# Functions

In [2]:
# given 5 vertex of the polygon find the one that is concave 
# the sum of internal angle of a polygon is (n - 2)* 180 
# convcave angle measure between 180 and 360 
def find_concave_angle(contours):
    # read clockwise order
    # need clock wise otherwise i am measuring the external angle 
    # I want to get the internal angle
    clockwise = contours[::-1]
    
    # take first and last 
    v1 = clockwise[-1] - clockwise[0]
    v2 = clockwise[1] - clockwise[0]
    # if cross product is negative means that angle is between 180 and 360 so it is the concave one 
    # |a| x |b| = |a||b| sin(teta) n
    # magnitude is always positive so only the sin(teta) give the sign sice n is just the unit vector
    # so if is negative we know that the angle is betwenn 180 and 360 
    # np.cross(a,b)  In cases where both input vectors have dimension 2, 
    # the z-component of the cross product is returned
    
    if np.cross(v1, v2) < 0:
            #vertex cordinate, index 
            return clockwise[0], 0
    for i in range(1 ,len(contours) - 1):
        v1 = clockwise[i-1] - clockwise[i]
        v2 = clockwise[i+1] - clockwise[i]
        
        if np.cross(v1, v2) < 0:
            return clockwise[i], i
    # case not find any concave vertex
    return None, None

In [3]:
# function to see if it is calculate right the angle 
# just a debug functions
def calcualte_angle(v1, v2):
    cross = np.cross(v1,v2)
    len_b = np.linalg.norm(v1)
    len_c = np.linalg.norm(v2)
    rad = np.arcsin(cross/ (len_b * len_c))
    deg = np.degrees(rad)
    #check if the cross product is positive, if dont, 
    # the angle must be adjusted to 360-angle
    # the negative indicate the direction in which the angle is meausure
    if deg < 0:
        print(360 - deg)
    else:
        print(deg)

In [4]:
# read the binary code 
def read_binary_code(contour, concave_index, otsu_img):
    # read clockwise
    clockwise = contour[::-1]
    # two vertex we are interested in
    # are always the second and the third one respect the concave vertex
    v1 = clockwise[((concave_index + 2) % 5)]
    v2 = clockwise[((concave_index + 3) % 5)]
    # find the point in the middle of the segment v1, v2
    v6 = v1 * (0.5) + v2 * (0.5)
    number = 0
    # calculate position for the marker 
    for i in range(1,6):
        # black mean 1
        # position where to look for the dot marker
        # 0.166 = 1 / 6 = 1 /( d + 1 ) = lenght of each uniform piece along the line
        pos = v6 * (0.166 * i ) + clockwise[concave_index]  * (1 - (0.166 * i))
        # if black dot
        # recall image is seen as matrix so x and y axis are opposite respect the cartesian 
        if otsu_img[int(pos[0][1]),int(pos[0][0])] == 0:
            number += np.power(2,i - 1)
    return number

In [5]:
def feautures_detection(frame_gray):
    # apply thresholding to convert the grayscale image to a binary image
    #  the function determines the optimal threshold value using the Otsu's
    ret,thresh = cv.threshold(frame_gray,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    # find the contours
    # For better accuracy, use binary images. So before finding contours, apply threshold
    #   param:vinary images, second is contour retrieval mode, third is contour approximation method
    #  Each contour is stored as a vector of points
    # cv.CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments and leaves only their end points
    contours,hierarchy = cv.findContours(thresh, cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    
    # list containing all corner features
    corner_feautures = []
    # concave vertex 
    concave_list = []
    # list containing all the number decoded
    number_list = []
  
    # for each countout find by cv.findContours
    for c in contours:

        # define the precision 0.11 0.13 0.14 0.17 a bit better
        epsilon = 0.017 * cv.arcLength(c,True)
        # approximate the contour
        # it aim to simplify the polyline by reducing the number of vertices based on epsilon value 
        approx = cv.approxPolyDP(c,epsilon,True)
        # computing area, helps to filter the countour not right for us
        area = int(cv.contourArea(approx))
        
        # draw the aproximation
        # if is a polygon with 5-6 vertex    area must be around that range  to be sure to take only polygon in the plate
        if len(approx) in range(5,7) and area in range(1000,8649) and approx[0][0][0] in range(1100, 1600):

            # i have to define better the countorn
            if len(approx) == 6:
                # 7.641
                approx = cv.approxPolyDP(approx,8.641,True)
                # refine the corners to subpixel accuracy
                approx = cv.cornerSubPix(thresh,np.float32(approx), (5,5),(-1,-1),(cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.001))
                concave_point, index_concave_vertex = find_concave_angle(approx)
               

            # case 5 vertex 
            else:
                # refine the corners to subpixel accuracy
                approx = cv.cornerSubPix(thresh,np.float32(approx), (5,5),(-1,-1),(cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.001))
                # find the concave point
                concave_point, index_concave_vertex = find_concave_angle(approx)
                
               

            # check concave vertex found it
            if index_concave_vertex != None and len(approx) == 5:
                # polygon counturn with concave vertex
                corner_feautures.append(approx)
                # decode binary encoding
                number = read_binary_code(np.int32(approx), index_concave_vertex, thresh)
                # decoded number
                number_list.append(number)
                # concave vertex 
                clockwise = approx[::-1]
                concave_list.append(clockwise[index_concave_vertex])
               
   
    # vertex index
    p0 = np.array(corner_feautures, dtype = np.float32)
    # concave vertices
    concave_vertices = np.array(concave_list, dtype = np.float32 )
    #counturn vertecises, concave vertices, number decoded
    return p0, concave_vertices, number_list

In [6]:
# filter polygon point 
# we need to dont propagate point that are detected with optical flow but then are behind the glass
# we filter even polygon point detected by that are to close to the glass exit
# because sometime they are not so precise
def filter_polygon(p, flag):
    # reshape
    p = np.array(np.reshape(p, (-1, 5, 2)))
    # save index to delete
    to_delete = []
    # case exit from glass
    if flag:
        for enum, cordinate  in enumerate(zip(p[:,:,0], p[:,:,1])):
             # check the range for x 
            x = np.where((cordinate[0]< 1207. ), True, False)
            # if any of thr polygon vertex are in the area we have to delete all of them
            if x.any():
                to_delete.append(enum)
         # delete poygon detected
        x = np.delete(p, to_delete, 0)  
        # reshape 
        x =  np.reshape(x,(-1,2))
        return x
    
    # case enter in the glass
    else:
        #                                   x cordinate y cordinate 
        for enum, cordinate  in enumerate(zip(p[:,:,0], p[:,:,1])):
            # check the range for x 
            x = np.where((cordinate[0]< 1250. ), True, False)
            # check the range for y 
            y = np.where(( cordinate[1] > 550. ) & ( cordinate[1] < 750. ), True, False)

            # if at least a true i have to delete the polygon 
            if x.any() & y.any():
                # print(f'enum {enum}')
                to_delete.append(enum)

        # delete poygon detected
        x = np.delete(p, to_delete, 0)  
        # reshape 
        x =  np.reshape(x,(-1,2))
        return x

In [7]:
# filter concave vertex that are detected with optical flow but then are behind the object 
# so we have to remove them 
# we filter even vertex point detected but that are to close to the glass exit
# because sometime they are not so precise
def filter_concave_vertex(concave_vertex, number, flag):
    # case exit the glass
    if flag:
        # third check x 
        third_range = np.argwhere(concave_vertex[:,0] < 1235. )
         #delete value that respect the condtion 
        filtered_concave_vertex = np.delete(concave_vertex,third_range,0)
        # reshape
        concave_vertex =  np.reshape(filtered_concave_vertex,(-1,2))
    
        # filter number decoded
        number = np.delete(number, third_range,0)
        return concave_vertex, number
    # case enter the glass
    else:
        # first check y
        first_range = np.argwhere(concave_vertex[:,1] >  550.) 
        # second check y
        second_range = np.argwhere(concave_vertex[:,1] < 777.) 
        # third check x 
        third_range = np.argwhere(concave_vertex[:,0] < 1250. )

        # verify multiple condition
        x = np.intersect1d(first_range, second_range)
        z = np.intersect1d(x, third_range)

        #delete value that respect the condtion 
        filtered_concave_vertex = np.delete(concave_vertex,z,0)
        # reshape
        concave_vertex =  np.reshape(filtered_concave_vertex,(-1,2))

        # filter number decoded
        number = np.delete(number, z,0)
        return concave_vertex, number

In [8]:
def draw_frame(polygon_vertex, concave_vertex, number, frame):
    # five point of the polygon
    for p in polygon_vertex:
        img = cv.drawMarker(frame, (int(p[0]), int(p[1])), color=[0,0,255], thickness=1, 
        markerType= cv.MARKER_CROSS , line_type=cv.LINE_AA)

    # print number and concave vertex
    for p,n in zip(concave_vertex, number):
        img = cv.drawMarker(frame, (int(p[0]), int(p[1])), color=[255,0,0], thickness=1, 
        markerType= cv.MARKER_CROSS , line_type=cv.LINE_AA)
         # write number decode 
        img = cv.putText(img = img, text = str(n), org =(int(p[0]), int(p[1])  ),fontFace = cv.FONT_HERSHEY_SIMPLEX, fontScale = 1, color =(0, 255, 0),
               thickness = 2)
    return img

In [9]:
def main(file_name):
    path = "Data/" + file_name + ".mp4"
    
    # lucas kanade params
    lk_params = dict(winSize = (10,10),
                    maxLevel = 2,
                    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

    # read first frame
    vid = cv.VideoCapture(path)
    # Take first frame and find corners in it
    ret, old_frame = vid.read()
    # lukas kanade works with grayscale images
    # convert the image to grayscale
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
    # find good features to pass to lukas kanade
    # p0: polygon vertex, concave_vertex : concave vertex, number: number decoded
    p0, concave_vertex, number = feautures_detection(old_gray)
    p0 = np.reshape(p0,(-1,2))
    concave_vertex_0 = np.reshape(concave_vertex,(-1,2))
    # frame count 
    frame_count = 0 
    
    
    
    while(vid.isOpened()):
        
        # read frame by frame
        ret, frame = vid.read()
        if not ret:
            print('No frames grabbed!')
            break

        # counting frame for refreh
        frame_count += 1
        # new frame 
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        # calculate optical flow              old frame,  new frame,  points, mask, param for lucas kanade
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, gray, p0 , None, **lk_params )
        # change size 
        p1 = np.reshape(p1,(-1,2))
        # filter the polygon vertex
        # since we are using lukas kanade optical flow and since at some point part of the marker refernce system is ocluded by the glass
        # some point the point behind the glass cannot be track anymore so we have to filter out them 
        p1 = filter_polygon(p1, False)
        # concave vertex 
        
        
        # use lukas kanade optical flow on the features we are interasting in
        # calculate optical flow              old frame,  new frame,  points, mask, param for lucas kanade
        concave_vertex_1, st, err = cv.calcOpticalFlowPyrLK(old_gray, gray,concave_vertex_0 , None, **lk_params )
        # change size 
        concave_vertex_1 = np.reshape(concave_vertex_1,(-1,2))
        # filter concave vertex that has been calculate with optical flow "entering the glass"
        concave_vertex_1, number = filter_concave_vertex(concave_vertex_1, number, False)
        
        
        
        # draw part
        frame = draw_frame(p1, concave_vertex_1, number, frame)
        
        cv.namedWindow("win1", cv.WINDOW_NORMAL )
        # display the image
        cv.imshow('win1', frame)
        
        
        
        # update frame
        old_gray = gray.copy()
        
        # every 20 frame
        # we recalculate the vertex polygon point the concave vertex and the decoded number 
        if frame_count % 20 == 0:
            p0, concave_vertex_0, number = feautures_detection(old_gray)
            p0 = np.reshape(p0,(-1,2))
            # we choose to use only the set of ploygon vertex that belong to a predefined range of cordinate in the image
            # sometime the point polygon close to the "glass exit" are not well defined by cv.approxPolyDP function so we 
            # choose to filtered them out
            p0 = filter_polygon(p0, True)
            concave_vertex_0 = np.reshape(concave_vertex_0,(-1,2))
             # filter concave vertex 
            concave_vertex_0, number = filter_concave_vertex(concave_vertex_0, number, True)
       
        else:
            # update old points
            p0 = p1.copy()
            concave_vertex_0 = concave_vertex_1.copy()

            
        if cv.waitKey(500) & 0xFF == ord('q'):
            break

    # After the loop release the vid object
    vid.release()
    # Destroy all the windows
    cv.destroyAllWindows()

# Main

In [11]:
# available video sequence:
#          0:parrot,     1:dinosaur,   2:craker,   3:budha statue 
file_name = ["obj01", "obj02", "obj03", "obj04"]
# video you want to run 
main(file_name[0])